<a href="https://www.kaggle.com/mohammedhamdy98/mri-classification-by-mobilenet?scriptVersionId=88004980" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
train_path = '../input/brain-tumor-mri-classification/MRICopy/train'
test_path = '../input/brain-tumor-mri-classification/MRICopy/test'
val_path = '../input/brain-tumor-mri-classification/MRICopy/val'

In [ ]:
train_batches = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
train_path, target_size=(224, 224), batch_size=2210)

test_batches = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
test_path, target_size=(224, 224), batch_size=560)

val_batches = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
val_path, target_size=(224, 224), batch_size=820)

In [ ]:
X_train, y_train = next(train_batches)
X_test, y_test = next(test_batches)
X_val, y_val = next(val_batches)

In [ ]:
test_batches.image_shape

In [ ]:
labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

In [ ]:
plt.figure(figsize=(20, 20))
for n, i in enumerate(np.random.randint(0, len(X_test), 10)):
    plt.subplot(5, 5, n+1)
    plt.imshow((X_test[i]*255).astype(np.uint8))
    plt.xlabel(labels[np.argmax(y_test[i])])

In [ ]:
plt.figure(figsize=(20, 20))
for n, i in enumerate(np.random.randint(0, len(X_train), 10)):
    plt.subplot(5, 5, n+1)
    plt.imshow((X_train[i]*255).astype(np.uint8))
    plt.xlabel(labels[np.argmax(y_train[i])])

In [ ]:
plt.figure(figsize=(20, 20))
for n, i in enumerate(np.random.randint(0, len(X_val), 10)):
    plt.subplot(5, 5, n+1)
    plt.imshow((X_val[i]*255).astype(np.uint8))
    plt.xlabel(labels[np.argmax(y_val[i])])

In [ ]:
img_train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = img_train_datagen.flow(X_train, y_train, batch_size=30)
valid_generator = img_train_datagen.flow(X_val, y_val)

In [ ]:
mobile_net = tf.keras.applications.mobilenet.MobileNet(dropout=0.4)

In [ ]:
model_mobile = tf.keras.Sequential()
for layer in mobile_net.layers[:-1]:
    model_mobile.add(layer)

In [ ]:
for layer in model_mobile.layers:
    layer.trainable = False

In [ ]:
model_mobile.add(tf.keras.layers.Dense(4, activation = 'Softmax'))

In [ ]:
model_mobile.summary()

In [ ]:
model_mobile.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

In [ ]:
history = model_mobile.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=40, batch_size = 60
                           , verbose=1, callbacks=[reduce_lr, early_stopping])

In [ ]:
predictions = model_mobile.predict(X_test, verbose=0)

In [ ]:
acc = model_mobile.evaluate(X_test, y_test)
acc

In [ ]:
plt.figure(figsize=(20, 20))
for n, i in enumerate(np.random.randint(0, len(predictions), 20)):
    plt.subplot(5, 5, n+1)
    plt.imshow((X_test[i]*255).astype(np.uint8))
    plt.xlabel(labels[np.argmax(y_test[i])])

In [ ]:
conf = confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))

In [ ]:
sns.heatmap(conf, annot= True, fmt='d')

In [ ]:
clas = classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1))
print(clas)

In [ ]:
plt.plot(history.history['val_loss'], color='red', label='val_loss')
plt.plot(history.history['loss'], color='blue', label='train_loss')
plt.xlabel('Epochs')
plt.ylabel("Loss")
plt.legend()

In [ ]:
plt.plot(history.history['val_accuracy'], color='red', label='val_accuracy')
plt.plot(history.history['accuracy'], color='blue', label='train_accuracy')
plt.xlabel('Epochs')
plt.ylabel("acc")
plt.legend()

In [ ]:
def prepare(path):
    img_arr = cv2.imread(path)
    img = cv2.resize(img_arr, (224, 224))
    img = np.expand_dims(img, axis=0)
    return img

In [ ]:
pred = model.predict([prepare('image path')])
print(labels[np.argmax(pred)])